# Overview

Previously we have seen examples of running the k-means algorithm provided by both scikit-learn and MLlib. In the previous notebook regarding [Running MLlib Algorothms](Running%20MLlib%20Algorithms%20%28k-means%29.ipynb) we learned that the MLlib algorithms are parallelized. This is a very useful feature as it allows us to shorten the amount of time to train/test a model on a large data set.

The problem exists however with the Spark implimentation; it does not support nested parallelism.

## Spark Doesnt Support Nested Parallelism

What we mean to say is that Apache Spark doesn't support any form of nesting in terms of spark managed parallelism. Distributed operations can be initialized only by the driver (ie. not in a worker process created by the driver). Having looked at the source code there are a few reasons for this. The main reason is that Sparks code checks and prevents a SparkContext from being created on a worker. Without access to the SparkContext we do not have access to distributed data structures, like Spark DataFrame, and execution of parallel processing functions, like training an MLlib algorithm.

If you tried to have a Spark Executor (worker) create and train an MLlib algorithm you would see the following error pop up:

```
AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?
```

This took me a while to figure out, but it is pointed out in this [discussion](https://coderedirect.com/questions/310003/run-ml-algorithm-inside-map-function-in-spark)

This is inconvenient as spark provides a number of useful mechanism for kicking off parallel processes based on our dataset. For example, in the previous notebook using scikit-learn models, we use the `.groupby(date).apply(training_function)` functionality to kick off a training job for each group of data in parallel. But unfortunayely, doe to the limitations noted above, we are not able to use this api with mlib. Again, this is because the training  function is executed on the worker where a spark context is not found and cannot be created.

### Options For Home Grown Nested Parllalelism

Ok, so we cant use a single SparkContext to provide nested parallelism, what are our options? Based on my understanding, I suspect the following two alternatives are possible (we prove one out in this notebook, the other is for a rainy day):
1. Have the driver kick off and manage parralel operations using a single SparkContext
2. Have the driver kick off and manage parralel operations using multiple SparkContexts pointing to multiple clusters
3. Hack the worker to create a SparkContext / Submit a PR to Spark's official Repo

**Note**: If you have other suggestions, please open a github issue so we can chat!

#### Option 1: Driver Manages Parallel Calls To SparkContext
We can do this using either multithreading or multiprocessing. If you are not familiar with these topics, please review the [prepared materials](../../../Programming/Parallelization/README.md)

Ok, so what are we building? We will be creating a function which will launch multiple threads, tell them to do work in parallel, collect the results, and assemble the result rest into one big result. Sometimes this is called "map reduce" sometimes its called "divide and conquer".

## There Be Dragons Ahead
**Note**: This is a very complex subject and requires engineering skills as well as knowledge of troubleshooting spark. See gotchas section.

One could make the case that we are abusing Spark (ie. using it in a way it was not designed). As a systems engineer that's basically my job so Nuts to that. With most distributed systems we will run into problems that we cannot explain or cannot reproduce. While writing this notebook I documented some issues I ran into and their solutions.

You might run out of memory when you have multiple jobs competing against eachother. 

```
Py4JJavaError: An error occurred while calling o689322.createOrReplaceTempView.
: java.lang.OutOfMemoryError: Java heap space
```

As such you may need to adjust your sparkConf to allocate more memory to the driver/worker.
```python
    sparkConf.set("spark.executor.instances", "3")
    sparkConf.set("spark.executor.cores", "2")
    sparkConf.set("spark.executor.memory", "4096m")
    sparkConf.set("spark.executor.memoryOverhead", "1024m")
    sparkConf.set("spark.driver.memory", "1024m")
```

Thing blowing up may cause other things to blow up unexpectedly. I coded a bug into my solution where i was killing a job before it finished writing data which corrupted the data file it was writing... Don't do this.

```
Py4JJavaError: An error occurred while calling o436701.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 22489.0 failed 4 times, most recent failure: Lost task 3.3 in stage 22489.0 (TID 311712) (10.42.0.1 executor 2): java.io.EOFException: Cannot seek after EOF
```

I encountered some other issues and wrapped my function in retry logic just incase (this may not have been needed as the issue might have been unavoidable and also resolved in a later version of code).

## Prerequisites

It assumes you have already read the following notebooks:
- [Install Apache Spark Prerequisites](Install%20Apache%20Spark%20Prerequisites.ipynb)
- [Spark Pi - The Hello World Example For Apache spark](Spark%20Pi%20-%20The%20Hello%20World%20Example%20For%20Apache%20spark.ipynb)
- [Intro To Koalas](Intro%20To%20Koalas.ipynb)
- [K-Means](../../Algorithms/Unsupervised%20Learning/Cluster%20Analysis/K-Means.ipynb)
- [Load CSV Into Apache Spark On Kubernetes](Load%20CSV%20Into%20Apache%20Spark%20On%20Kubernetes.ipynb)

The instructions are basically the same as [Running MLlib Algorothms](Running%20MLlib%20Algorithms%20%28k-means%29.ipynb) with the added framework I mentioned.

## Adjenda
1. Create SparkContext
2. Create Web Server To Host Data
3. Load The Data
4. Develop Parallelization Framework
5. Test Framework Performance
8. Cleanup

# 1. Create SparkContext

In [1]:
import pyprojroot
project_root_dir  = pyprojroot.here()
print(project_root_dir)

/ml-training-jupyter-notebooks


In [2]:
spark_app_name = "spark-jupyter-mlib"
docker_image = "tschneider/apache-spark-k8:v7"
k8_master_ip = "15.4.7.11"

import spark_helper
spark_session = spark_helper.create_spark_session(spark_app_name, docker_image, k8_master_ip)
sc = spark_session.sparkContext

Setting SPARK_HOME
/usr/lib/spark-3.1.1-bin-hadoop2.7

Running findspark.init() function
['/usr/lib/spark-3.1.1-bin-hadoop2.7/python', '/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip', '/ml-training-jupyter-notebooks/Machine Learning/Big Data And Big Compute/Apache Spark', '/usr/local/lib/python39.zip', '/usr/local/lib/python3.9', '/usr/local/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/site-packages', '/ml-training-jupyter-notebooks/Utilities']

Setting PYSPARK_PYTHON
/usr/local/bin/python3

Configuring URL for kubernetes master
k8s://https://15.4.7.11:6443

Determining IP Of Server
The ip was detected as: 15.4.12.12

Creating SparkConf Object
('spark.master', 'k8s://https://15.4.7.11:6443')
('spark.app.name', 'spark-jupyter-mlib')
('spark.submit.deploy.mode', 'cluster')
('spark.kubernetes.container.image', 'tschneider/apache-spark-k8:v7')
('spark.kubernetes.namespace', 'spark')
('spark.kubernetes.pyspark.pythonVersion', '3')
('spark.kubernetes.authenti

22/01/17 21:50:43 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 15.4.12.12 instead (on interface eth0)
22/01/17 21:50:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/17 21:50:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).



Done!


In [3]:
! kubectl -n spark get pods

NAME                                         READY     STATUS    RESTARTS   AGE
spark-jupyter-mlib-169d3f7e6a05714e-exec-1   1/1       Running   0          23s
spark-jupyter-mlib-169d3f7e6a05714e-exec-2   1/1       Running   0          22s
spark-jupyter-mlib-169d3f7e6a05714e-exec-3   1/1       Running   0          21s


# 2. Setup Datastore

In [5]:
import os
data_dir_name = "Example Data Sets"
data_dir_path = os.path.join(project_root_dir, data_dir_name)
spark_helper.link_data_dir_to_root(data_dir_path)

In [6]:
# Import our custom module for the web server we wrote
import PythonHttpFileServer

In [7]:
import os

data_dir_name = "Example Data Sets"
web_root = os.path.join(project_root_dir, data_dir_name)

if not os.path.exists(web_root):
    raise Exception("The web root for the server does not exist.")

csv_file_name = "nasdaq_2019.csv"
csv_file_path = os.path.join(web_root, csv_file_name)

if not os.path.exists(csv_file_path):
    raise Exception("The data file does not exist.")
    
print("Web root and data file exist!")
print("web root: {0}".format(web_root))
print("data file: {0}".format(csv_file_path))

Web root and data file exist!
web root: /ml-training-jupyter-notebooks/Example Data Sets
data file: /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv


In [8]:
# Import the library
import threading

# Configure the logger and log level (incase we need/want to debug)
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create and start the thread if it doesnt exist
var_exists = 'web_server_thread' in locals() or 'web_server_thread' in globals()
if not var_exists:
    web_server_port = 80
    web_server_args = (web_server_port, web_root)
    web_server_thread = threading.Thread(target=PythonHttpFileServer.run_server, args=web_server_args)
    web_server_thread.start()
else:
    print("Web Server thread already exists")
    print("To kill it you need to restart the kernel.")

INFO:root:Starting server on port 80
INFO:root:Web root specified as: /ml-training-jupyter-notebooks/Example Data Sets


 * Serving Flask app 'PythonHttpFileServer' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://15.4.12.12:80/ (Press CTRL+C to quit)


# 3. Load The Data 
To help debug/test our framework we will load the data and build a few utilities targeting pices of the data. Once everything works we will run everything in parallel. Again, this step is just for Testing.

## 3.1. Add File To Spark Cluster

In [9]:
ip_address = spark_helper.determine_ip_address()
csv_file_name = "nasdaq_2019.csv"
csv_file_url = "http://{0}:{1}/{2}".format(ip_address, web_server_port, csv_file_name)
print("Uploading file '{0}' to Spark cluster.".format(csv_file_url))
sc.addFile(csv_file_url)

INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv
INFO:werkzeug:15.4.12.12 - - [17/Jan/2022 21:51:35] "GET /nasdaq_2019.csv HTTP/1.1" 200 -


Uploading file 'http://15.4.12.12:80/nasdaq_2019.csv' to Spark cluster.


## 3.2. Use Koalas To Load Data File Into DataFrame

Import the utility function to convert a date string to a datetime object from our utilities module

In [10]:
# Import the utilities module we wrote
import utilities

# Define a mapping to convert our data field to the correct type
converter_mapping = {
    "date": utilities.convert_date_string_to_date
}

Load our OHCLV data Into a koalas dataframe and pull out a single day in the say way we would in pandas

In [11]:
# Avoid a warning
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ["SPARK_KOALAS_AUTOPATCH"] = "0"

from databricks import koalas
koalas_dataframe = koalas.read_csv(u"file:////nasdaq_2019.csv", converters=converter_mapping)

INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv
INFO:werkzeug:15.4.7.102 - - [17/Jan/2022 21:51:47] "GET /nasdaq_2019.csv HTTP/1.1" 200 -
INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv  
INFO:werkzeug:15.4.7.103 - - [17/Jan/2022 21:51:51] "GET /nasdaq_2019.csv HTTP/1.1" 200 -
INFO:root:Get /ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv6]
INFO:werkzeug:15.4.7.101 - - [17/Jan/2022 21:51:51] "GET /nasdaq_2019.csv HTTP/1.1" 200 -


In [12]:
koalas_dataframe.head()

22/01/17 21:52:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/17 21:52:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,ticker,interval,date,open,high,low,close,volume
0,AABA,D,2019-07-01,70.90,71.5200,70.3250,70.57,10234800
1,AAL,D,2019-07-01,33.14,33.6632,32.5301,32.88,8995100
2,AAME,D,2019-07-01,2.43,2.4300,2.4000,2.40,500
3,AAOI,D,2019-07-01,10.70,10.8900,10.0100,10.18,883100
4,AAON,D,2019-07-01,50.57,50.9850,48.5600,49.73,180200


# 4. Develop Parralelization Framework

Our objective is to apply the kmeans algorithm from Spark MLlib to each date in our koalas_dataframe object. Because the data contain mutually exclusive data, we can parallelize accross this variable.

To do this, we leverage a parallelization utility we wrote previously. We can do this using either multithreading or multiprocessing. If you are not familiar with these topics, please review the [prepared materials](../../../Programming/Parallelization/README.md).

We will talk more about this in a second. For the moment, we need to write the function which we will parallelize.

## 4.1. Create Utility Function


We then write and test our function which performs kmeans on a specific date

In [13]:
import pandas
import pyspark
from pyspark.sql.functions import pandas_udf, udf
from pyspark.sql.types import *
from databricks import koalas
from pyspark.ml.clustering import KMeans

koalas.set_option("compute.ops_on_diff_frames", True)

def perform_kmeans_on_dataframe(date, column_names):
    
    # Load the data and filter for the data of interest
    tmp = koalas.read_csv(u"file:////nasdaq_2019.csv", converters=converter_mapping)
    tmp = tmp.loc[tmp["date"] == date]
    
    # Create our model
    model = KMeans().setK(5).setSeed(42)

    # Do some magic to get the data in the right format for the spark model
    from pyspark.ml.feature import VectorAssembler
    assembler = VectorAssembler(inputCols=column_names, outputCol="features")
    if type(tmp) == koalas.frame.DataFrame:
        model_parameters = assembler.transform(tmp[[*column_names]].to_spark())
    elif type(tmp) == pandas.DataFrame:
        tmp = koalas.DataFrame(tmp)
        model_parameters = assembler.transform(tmp[[*column_names]].to_spark())

    # Train the model
    trained_model = model.fit(model_parameters)
    
    # Extract the cluster information for the training data
    predictions = trained_model.transform(model_parameters)
    cluster_indices = predictions.select("prediction")
    cluster_indices = koalas.DataFrame(cluster_indices).to_numpy().reshape(-1)
    cluster_indices = koalas.Series(cluster_indices, index=tmp.index.to_numpy())
    tmp["cluster_indices"] = cluster_indices
    cluster_centroids = trained_model.clusterCenters()
    tmp["cluster_centroids"] = tmp["cluster_indices"].apply(lambda i: str(cluster_centroids[i]))

    return tmp


In [14]:
perform_kmeans_on_dataframe('2019-01-02', column_names=["open", "close"]).head()

22/01/17 21:53:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/01/17 21:53:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/01/17 21:53:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/17 21:53:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/17 21:53:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
/usr/local/lib/python3.9/site-packages/pyspark/sql/pandas/functions.py:389: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.


,ticker,interval,date,open,high,low,close,volume,cluster_indices,cluster_centroids
96829,ACLS,D,2019-01-02,17.48,18.14,16.920,17.79,284200,0,[12.97707127 13.2648608 ]
96923,ALLK,D,2019-01-02,51.26,53.94,50.115,51.99,151900,3,[66.74787778 67.59067778]
97216,BRPAU,D,2019-01-02,10.55,10.55,10.550,10.55,0,0,[12.97707127 13.2648608 ]
97658,DWFI,D,2019-01-02,22.21,22.24,22.200,22.20,41300,0,[12.97707127 13.2648608 ]
97699,EFAS,D,2019-01-02,15.08,15.08,15.080,15.08,0,0,[12.97707127 13.2648608 ]


**Note**: The warning above is coming from code used in the internals of Koalas. Do not worry about this warning.

## 4.2. Prepare Parallelization Inputs
Now that the utility function has been tested on smaller dataframes, we are safe to run it on a large data set in parrallel. 

In [25]:
# Get a list of the dates in our dataframe
dates = koalas_dataframe["date"].unique().sort_values().to_numpy()
dates[0]

22/01/17 21:35:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


'2019-01-01'

In [15]:
# Delete the dataframe which is no longer needed and free up spark resources
if 'koalas_dataframe' in locals() or 'koalas_dataframe' in globals():
    del koalas_dataframe

We will need to build some utilities to facilitate the parralization of these opersations. As we mentioned in the Gotchas section.

## 4.3. Create Parallelization Functions

We previousle wrote a function which will perform kmeans on a data. Now we need to go a step further and record the results to an aggregate output file. This output file will later be loaded to show the results for all dates.

In [28]:
import time

# Define a function to perform kmeans and store the results in a local data file

def parallel_function(params):
    
    # Retrieve params
    date = params[0]
    lock = params[1]
    result_file_path = params[2]
    
    # Do the computation
    result_df = perform_kmeans_on_dataframe(
        date, 
        column_names=["open", "close"])
    
    # Force spark to not be lazy and to do the computation
    result_df.shape  
    
    # Record the results to a local data file
    with lock:
        if os.path.exists(result_file_path):                  
            result_df.to_pandas().to_csv(result_file_path, mode='a', index=False)
        else:
            result_df.to_pandas().to_csv(result_file_path, mode='a', index=False, header=False)

    # Return results and timing info to the ThreadHelper
    return result_df

# Write a function to retry the code if an error is encountered.
# We may enconuter some weird transient errors when runnong on spark
# As I mentioned earlier "there be dragons"
# So we have a simply retry wrapper to handle any weird execptions that 
# can be resolved by rerunning the same code

def retry_wrapper(params, max_retries=5):
    retries_remaining = max_retries
    while True:
        try:
            return parallel_function(params)
        except Exception as e:
            retries_remaining -= 1
            if retries_remaining > 0:
                print("Error occurred with parameter set: {0}. {1} retries remaining".format(params, max_retires - retries_remaining))
                time.sleep(1)
            else:
                raise e

We can test that this works with:

In [29]:
# Determine path to resutls file
def get_result_file_path():
    project_root_dir  = pyprojroot.here()
    result_file_name = "results.csv"
    result_file_path = os.path.join(project_root_dir, "Example Data Sets", result_file_name)
    return result_file_path

# Assemble parameters for function
import threading
date = '2019-01-01'
lock = threading.Lock()
result_file_path = get_result_file_path()
params = [date, lock, result_file_path]

# run function
result_df = retry_wrapper(params)
result_df.head()

# Cleanup
os.r(result_file_path)

22/01/17 22:23:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/17 22:23:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/17 22:23:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/17 22:23:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/17 22:24:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


AttributeError: module 'os' has no attribute 'delete'

Write a function to kick things off in parrallel and returns the results. The trick here is that the results file is going to be stored in our example datasets folder and served to the workers via our web server.

In [27]:
# Create a ThreadPool and kick off the parrallel training sessions
import os
import itertools
from datetime import datetime
import threading
import parallelization

def run_multithreaded_kmeans(dates):
    try:
        print("Create vars to help with synchronization and parallelization")
        mutex = threading.Lock()
        num_threads = 10
        thread_pool = parallelization.EnhancedThreadPool(num_threads)
            
        print("Create result file to store results from threads")
        project_root_dir  = pyprojroot.here()
        result_file_name = "results.csv"
        result_file_path = os.path.join(project_root_dir, "Example Data Sets", result_file_name)
        do_work = False
        if not os.path.exists(result_file_path):          
            with open(result_file_path, 'w') as fp:
                pass
            do_work = True
        else:
            print("No work to do, results file already exists!")
        
        print("Setting up local datastore for driver")
        data_dir_name = "Example Data Sets"
        data_dir_path = os.path.join(project_root_dir, data_dir_name)
        spark_helper.link_data_dir_to_root(data_dir_path)

        if do_work:
            print("No result file exists, doing work...")
            start = datetime.now()
            print("Starting: {0}".format(start))

            print("Create an iterator of params for the thread function")
            num_ops = len(dates)
            input_file_name = "nasdaq_2019.csv"
            input_file_path = "file:///{0}".format(input_file_name)
        
            iterator = zip(dates, 
                           itertools.repeat(bar), 
                           itertools.repeat(mutex),
                           itertools.repeat(input_file_path),
                           itertools.repeat(result_file_path))
            
            print("Adding data file to cluster")
            sc.addFile(csv_file_url)
            
            print("Run training sessions for each data in parrallel")
            results = thread_pool.parallelize(thread_wrapper__func, iterator)

            # Record the end time
            calc_end = datetime.now()
            calc_diff = (calc_end - start).total_seconds()
            print("Ending: {0}".format(calc_end))
            print("Total calculation time: {0}s".format(calc_diff))
            date_diff = calc_diff / num_ops
            print("Time per date: {0}s".format(date_diff))

        print("Load results file")
        ip_address = spark_helper.determine_ip_address()
        web_server_port = 80
        result_file_url = "http://{0}:{1}/{2}".format(ip_address, web_server_port, result_file_name)
        worker_result_file_path = "file:///{0}".format(result_file_name)
        spark_helper.link_data_dir_to_root(data_dir_path)
        try:
            mutex.acquire()
            sc.addFile(result_file_url)
            merged_df = koalas.read_csv(worker_result_file_path, converters=converter_mapping)
        finally:
            mutex.release()
        
        return merged_df
        
    except Exception as e:
        # Cleanup spark
        sc.cancelAllJobs()
        # Raise error
        raise e
        

In [28]:
# Run the calculation
merged_df = run_multithreaded_kmeans(dates[0:4])

Create vars to help with synchronization and parallelization
Create result file to store results from threads
Setting up local datastore for driver
Creating Symlink: /ml-training-jupyter-notebooks/Example Data Sets/results.csv -> /results.csv
No result file exists, doing work...
Starting: 2022-01-17 21:38:05.092046
Create objects to help track multithreading progress


NameError: name 'create_progress_bar' is not defined

In [ ]:
# Show the union df
merged_df.loc[merged_df["date"] == dates[0]].head()

If anything went wrong, we should ask the SparkContext to kill any abandoned jobs that may be lingering in ghosted threads from the thread pool

In [ ]:
sc.cancelAllJobs()

# 5. Test The Framework
Now we can run a large set of dates using our threadpool

In [ ]:
spark_session.stop()

In [ ]:
spark_session = spark_helper.create_spark_session(spark_app_name, docker_image, k8_master_ip)
sc = spark_session.sparkContext

In [ ]:
results_file_path = os.path.join(data_dir_path, "results.csv")
os.remove(results_file_path)

In [ ]:
koalas.set_option("compute.ops_on_diff_frames", True)

In [ ]:
merged_df = run_multithreaded_kmeans(dates)

In [ ]:
merged_df.loc[merged_df["date"] == dates[7]].head()

If anything went wrong, we should ask the SparkContext to kill any abandoned jobs that may be lingering in ghosted threads from the thread pool

In [ ]:
sc.cancelAllJobs()

We see that computing every date is almost as fast as computing a single date.

## 5.3. Compare Results

We timed ourselves when running against a single day, a few days, and all days to show the operations are occurring in parrallel.

At first glance, it looks like the actual per date computation got faster. This is a bit strange. My only guess is that loading the data file was now spread accross multiple dates instead of one in the calculation.

97 seconds / 4 dates = 24.25 secondsper date
2605 seconds / 158 dates = 16.5 seconds per date

The important thing here is that we could scale our cluster out more and more nodes and the comutation should decrease.

# 6. Cleanup

In [ ]:
spark_helper.unlink_data_dir_from_root(data_dir_path)

In [ ]:
spark_session.stop()

In [ ]:
! kubectl -n spark get pod